In [108]:
#lets start by importing a bunch of stuff
import tensorflow as tf
import pandas as pd
import numpy as np
import math 

In [105]:
# Downloading and separating data

#explictily setting the types and names
names_data = ['entry','entry_name','protein_name','gene_name','organism','length','sequence',
              'gene_ontology','status','organism_id','keywords']

dtypes_data = {'entry':'str','entry_name':'str','protein_name':'str','gene_name':'str',
               'organism':'str','length':'int','sequence':'str','gene_ontology':'str','status':'str',
               'organism_id':'int','keywords':'str'}


protein_data = pd.read_csv('uniprot-filtered-reviewed_yes.tab',sep='\t',names = names_data,dtype=dtypes_data,skiprows=1)


#Seeing what the data looks like
protein_data.shape

#randomly shuffling dataset 

protein_data = protein_data.sample(frac=1)



In [106]:
#Ignoring protein's with ambigous amino acid codes 

protein_data = protein_data[~protein_data.sequence.str.contains('B')]
protein_data = protein_data[~protein_data.sequence.str.contains('O')]
protein_data = protein_data[~protein_data.sequence.str.contains('J')]
protein_data = protein_data[~protein_data.sequence.str.contains('U')]
protein_data = protein_data[~protein_data.sequence.str.contains('X')]
protein_data = protein_data[~protein_data.sequence.str.contains('Z')]

protein_data.shape


(556604, 11)

In [121]:
#Dividing into 80 percent training and 20 percent test
train, test = np.split(protein_data, [int(.8*len(protein_data))])

#removing GO labels and keywords from the test dataset
test = test.drop(columns=['gene_ontology','keywords'])




(111321, 9)